<a href="https://colab.research.google.com/github/peterbabulik/Quantum-Alice-Bob-Demo/blob/main/QuantumSafeStorage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 📦 The "Quantum Safe" (Self-Destructing Storage)
# @markdown ### 🟢 Experiment: Unclonable Storage
# @markdown This simulation enforces Quantum Laws on a file.
# @markdown 1. **Alice** locks a message in the safe.
# @markdown 2. **Eve** tries to hack it (Wrong Basis) -> **The Data Destroys Itself.**
# @markdown 3. **Bob** tries to read it (Right Basis) -> **He finds only noise.**

import numpy as np
import subprocess
import sys

# Install PennyLane
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
try:
    import pennylane as qml
except ImportError:
    install("pennylane")
    import pennylane as qml

class QuantumSafe:
    def __init__(self, message):
        self.num_qubits = len(message) * 8
        self.dev = qml.device("default.qubit", wires=1, shots=1)
        self.qubits = []
        self.bases = [] # The "Key" (kept secret)

        print(f"🔒 INITIALIZING SAFE with {self.num_qubits} qubits...")
        self._encrypt_data(message)

    def _prepare_qubit(self, bit, basis):
        """Internal physics engine to create state"""
        if bit == 1:
            qml.PauliX(wires=0)
        if basis == 1:
            qml.Hadamard(wires=0)
        return qml.state()

    def _encrypt_data(self, message):
        """Converts text to fragile quantum states"""
        # Convert text to bits
        msg_bits = []
        for char in message:
            bin_char = format(ord(char), '08b')
            msg_bits.extend([int(b) for b in bin_char])

        # Generate random bases (The Key)
        self.bases = np.random.randint(0, 2, len(msg_bits))

        # Store states
        # NOTE: In simulation, we store the vector.
        # But we will program the 'read' function to destroy it.
        dummy_dev = qml.device("default.qubit", wires=1)

        @qml.qnode(dummy_dev, interface='numpy')
        def get_state(bit, basis):
            if bit == 1:
                qml.PauliX(wires=0)
            if basis == 1:
                qml.Hadamard(wires=0)
            return qml.state()

        for i in range(len(msg_bits)):
            state = get_state(msg_bits[i], self.bases[i])
            self.qubits.append(np.array(state)) # Store the fragile state

        print("✅ Message locked inside Quantum Memory.")
        print(f"🔑 Secret Key (Bases): {self.bases[:10]}...")

    def attempt_unlock(self, guessed_bases):
        """
        Tries to read the data.
        If you guess the basis WRONG, the state COLLAPSES and is overwritten.
        """
        if len(guessed_bases) != len(self.qubits):
            return "❌ Error: Key length mismatch."

        readout_bits = []
        damage_report = 0

        print("\n⚙️  Attempting to unlock...")

        for i in range(len(self.qubits)):
            # 1. Check if the guess matches the reality
            # In real physics, we don't 'check'. We just measure.
            # If we measure in the wrong basis, we DESTROY the info.

            # Scenario A: Correct Basis (Measurement reveals truth)
            if guessed_bases[i] == self.bases[i]:
                # Measure in correct basis
                result = self._measure(self.qubits[i], guessed_bases[i])

            # Scenario B: Wrong Basis (Measurement destroys info)
            else:
                damage_report += 1
                # We measure in the WRONG basis
                result = self._measure(self.qubits[i], guessed_bases[i])

                # PHYSICS CONSEQUENCE:
                # The state is now collapsed to the wrong basis.
                # The original information is GONE.
                # We update the stored qubit to this new, useless state.
                self.qubits[i] = self._collapsed_state(result, guessed_bases[i])

            readout_bits.append(result)

        # Convert bits back to text (if possible)
        decoded = self._bits_to_text(readout_bits)

        print(f"💥 DAMAGE REPORT: {damage_report} qubits destroyed by wrong measurements.")
        return decoded

    def _measure(self, state, basis):
        """Simulates measurement and collapse"""
        @qml.qnode(self.dev)
        def meas(s, b):
            qml.StatePrep(s, wires=0)
            if b == 1: # Diagonal basis measurement
                qml.Hadamard(wires=0)
            return qml.sample(qml.PauliZ(0))

        res = int(meas(state, basis))
        # Map eigenvalue to bit
        return 0 if res == 1 else 1

    def _collapsed_state(self, bit, basis):
        """Generates the new 'ruined' state after a bad measurement"""
        dummy_dev = qml.device("default.qubit", wires=1)
        @qml.qnode(dummy_dev, interface='numpy')
        def prep(b, bas):
            if b == 1: qml.PauliX(wires=0)
            if bas == 1: qml.Hadamard(wires=0)
            return qml.state()
        return np.array(prep(bit, basis))

    def _bits_to_text(self, bits):
        chars = []
        for i in range(0, len(bits), 8):
            byte = bits[i:i+8]
            if len(byte) < 8: break
            try:
                char_code = int("".join(map(str, byte)), 2)
                if 32 <= char_code <= 126: # Printable ASCII
                    chars.append(chr(char_code))
                else:
                    chars.append('?') # Corrupted
            except:
                chars.append('?')
        return "".join(chars)


# ==========================================
# 🎮 THE SIMULATION
# ==========================================

# 1. Alice creates the safe
secret = "TOP_SECRET_PLANS"
safe = QuantumSafe(secret)

# 2. Eve tries to hack it (Random Guessing)
print("\n🦹 EVE attempts to hack the safe...")
# Eve generates random bases
eve_guess = np.random.randint(0, 2, safe.num_qubits)
eve_result = safe.attempt_unlock(eve_guess)

print(f"🦹 Eve's Result: '{eve_result}'")
print("(Notice it is mostly garbage because she guessed ~50% wrong)")

# 3. Bob tries to open it (With the CORRECT Key)
# BUT... Eve has already touched the qubits!
print("\n🕵️ BOB attempts to open the safe (using the Correct Key)...")
# Bob uses the correct bases that Alice gave him
bob_result = safe.attempt_unlock(safe.bases)

print(f"🕵️ Bob's Result: '{bob_result}'")
print("\n🚨 CONCLUSION: The data is corrupted for Bob too!")
print("   Because Eve measured the quantum state, she physically changed it.")
print("   This proves the file is 'Tamper-Evident'.")

/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pennylane/devices/device_api.py:193: PennyLaneDeprecationWarning: Setting shots on device is deprecated. Please use the `set_shots` transform on the respective QNode instead.
  warnings.warn(
/tmp/ipython-input-1022881385.py:122: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  res = int(meas(state, basis))


🔒 INITIALIZING SAFE with 128 qubits...
✅ Message locked inside Quantum Memory.
🔑 Secret Key (Bases): [0 0 0 0 1 1 0 0 1 0]...

🦹 EVE attempts to hack the safe...

⚙️  Attempting to unlock...
💥 DAMAGE REPORT: 64 qubits destroyed by wrong measurements.
🦹 Eve's Result: 'T??]?T??U????H?P'
(Notice it is mostly garbage because she guessed ~50% wrong)

🕵️ BOB attempts to open the safe (using the Correct Key)...

⚙️  Attempting to unlock...
💥 DAMAGE REPORT: 0 qubits destroyed by wrong measurements.
🕵️ Bob's Result: 'P??^?D?r??XY?I??'

🚨 CONCLUSION: The data is corrupted for Bob too!
   Because Eve measured the quantum state, she physically changed it.
   This proves the file is 'Tamper-Evident'.
